In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nfl-scores-and-betting-data/nfl_stadiums.csv
/kaggle/input/nfl-scores-and-betting-data/nfl_teams.csv
/kaggle/input/nfl-scores-and-betting-data/spreadspoke_scores.csv
/kaggle/input/nfl-scores-and-betting-data/spreadspoke.R


Below is the code used to obtain standings data for every week between 2012 and now

In [65]:
def clean_pfr_tables(input_df, afc_or_nfc):
    
    output_df = input_df.loc[~input_df.Tm.str.contains(afc_or_nfc)]
    output_df.Tm = output_df.Tm.str.replace('[^ 0-9a-zA-Z]+', '' ,regex=True)
    return output_df
    
def get_weekly_data(my_url):
    week_number = re.findall("(?:week=)(\d+)", my_url)[0]
    year_number = re.findall("(?:year=)(\d+)", my_url)[0]
    tables = pd.read_html(my_url)
    afc = tables[0]
    nfc = tables[1]
    
    cleaned_afc = clean_pfr_tables(afc, "AFC")
    cleaned_nfc = clean_pfr_tables(nfc, "NFC")
    
    combined_week = pd.concat([cleaned_afc, cleaned_nfc])
    
    combined_week['week_number'] = week_number    
    combined_week['year_number'] = year_number
    
    combined_week = combined_week.reset_index(drop=True)
    
    return combined_week

#weeks = list(range(1,19))
#years = list(range(2012,2022))
#df_list = []

#for year in years:
#    for week in weeks:
#        df_list.append(get_weekly_data(f'https://www.pro-football-reference.com/boxscores/standings.cgi?week={week}&year={year}&wk_league=NFL'))
        
#full_standings_data = pd.concat(df_list, axis=0, ignore_index = True)
#full_standings_data.to_csv("./full_standings_data.csv")

In [165]:
# Read already generated standings data

full_standings_data = pd.read_csv("./full_standings_data.csv").drop("Unnamed: 0", axis = 1)

full_standings_data[['Pts', 'PtsO', 'W', 'L', 'T', 'year_number', 'week_number']] = full_standings_data[['Pts', 'PtsO', 'W', 'L', 'T', 'year_number', 'week_number']].apply(pd.to_numeric, errors = 'coerce', axis=1)

full_standings_data['off_ppg'] = full_standings_data.Pts / (full_standings_data['W'] + full_standings_data['L'] + 
                                                                              full_standings_data['T'])

full_standings_data['def_ppg'] = full_standings_data.PtsO / (full_standings_data['W'] + full_standings_data['L'] + 
                                                                              full_standings_data['T'])

full_standings_data['rank_offense'] = full_standings_data.groupby(['year_number', 'week_number'])['off_ppg'].rank(method = 'first', ascending = False)
full_standings_data['rank_defense'] = full_standings_data.groupby(['year_number', 'week_number'])['def_ppg'].rank(method = 'first', ascending = True)

full_standings_data['rank_winpct'] = full_standings_data.groupby(['year_number', 'week_number'])['W-L%'].rank(method = 'first', ascending = False)


full_standings_data.head()

,Tm,W,L,T,W-L%,Pts,PtsO,PtDif,MoV,week_number,year_number,off_ppg,def_ppg,rank_offense,rank_defense,rank_winpct
0,New England Patriots,1.0,0.0,0.0,1.0,34.0,13.0,21,1.3,1.0,2012.0,34.0,13.0,6.0,3.0,1.0
1,New York Jets,1.0,0.0,0.0,1.0,48.0,28.0,20,1.3,1.0,2012.0,48.0,28.0,1.0,22.0,2.0
2,Miami Dolphins,0.0,1.0,0.0,0.0,10.0,30.0,-20,-1.3,1.0,2012.0,10.0,30.0,31.0,23.0,17.0
3,Buffalo Bills,0.0,1.0,0.0,0.0,28.0,48.0,-20,-1.3,1.0,2012.0,28.0,48.0,11.0,32.0,18.0
4,Baltimore Ravens,1.0,0.0,0.0,1.0,44.0,13.0,31,1.9,1.0,2012.0,44.0,13.0,2.0,4.0,3.0


Below is the code used to obtain preseason ratings data for every year between 2012 and now

In [129]:
def get_preseason_rankings(my_url, year):
    standings_data = pd.read_html(my_url)
    current_year = standings_data[0]
    current_year['season'] = year
    return current_year

#preseason_rankings_list = []
#years = list(range(2012,2022))

#for year in years:
#    preseason_rankings_list.append(get_preseason_rankings(my_url = f"https://www.teamrankings.com/nfl/rankings/teams/?date={year}-08-30", year = year))
    
#preseason_rankings_df = pd.concat(preseason_rankings_list, axis=0, ignore_index = True)
#preseason_rankings_df.to_csv("./preseason_rankings_df.csv")

In [130]:
# Read already generated preseason_ratings data

preseason_rankings_df = pd.read_csv("./preseason_rankings_df.csv")

In [136]:
preseason_rankings_df.dtypes
preseason_rankings_df.head()

,Team,Predictive,Home,Away,Last 5,In Div.,SOS,season
0,New England Patriots,1,1,1,24,24,1,2012
1,Pittsburgh Steelers,2,2,2,14,14,2,2012
2,New Orleans Saints,3,3,3,10,10,3,2012
3,Philadelphia Eagles,4,4,4,8,8,4,2012
4,Green Bay Packers,5,5,5,25,25,5,2012


In [5]:
teams = pd.read_csv('../input/nfl-scores-and-betting-data/nfl_teams.csv')
teams.head()

,team_name,team_name_short,team_id,team_id_pfr,team_conference,team_division,team_conference_pre2002,team_division_pre2002
0,Arizona Cardinals,Cardinals,ARI,CRD,NFC,NFC West,NFC,NFC West
1,Phoenix Cardinals,Cardinals,ARI,CRD,NFC,NaN,NFC,NFC East
2,St. Louis Cardinals,Cardinals,ARI,ARI,NFC,NaN,NFC,NFC East
3,Atlanta Falcons,Falcons,ATL,ATL,NFC,NFC South,NFC,NFC West
4,Baltimore Ravens,Ravens,BAL,RAV,AFC,AFC North,AFC,AFC Central


In [102]:
scores = pd.read_csv('../input/nfl-scores-and-betting-data/spreadspoke_scores.csv')
scores['schedule_week'] = pd.to_numeric(scores.schedule_week, errors = "coerce")
scores = scores.dropna(subset = ['schedule_week'])

# filter scores data
recent_scores = scores[scores["schedule_season"].isin(list(range(2012, 2022)))]

In [166]:


scores_and_standings = pd.merge(recent_scores, full_standings_data[['Tm', 'W', 'L', 'T', 'Pts', 'PtsO', 'week_number', 'year_number', 'off_ppg', 'def_ppg', 'rank_offense',
       'rank_defense', 'rank_winpct']].add_suffix("_home"), 
                                how = "left", left_on = ['schedule_season', 'schedule_week', 'team_home'], 
                                right_on=['year_number_home', 'week_number_home', 'Tm_home'])

scores_and_standings_all = pd.merge(scores_and_standings, full_standings_data[['Tm', 'W', 'L', 'T', 'Pts', 'PtsO', 'week_number', 'year_number', 'off_ppg', 'def_ppg', 'rank_offense',
       'rank_defense', 'rank_winpct']].add_suffix("_away"), 
                                how = "left", left_on = ['schedule_season', 'schedule_week', 'team_away'], 
                                right_on=['year_number_away', 'week_number_away', 'Tm_away'])

scores_and_standings_all.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,...,T_away,Pts_away,PtsO_away,week_number_away,year_number_away,off_ppg_away,def_ppg_away,rank_offense_away,rank_defense_away,rank_winpct_away
0,9/5/2012,2012,1.0,False,New York Giants,17.0,24.0,Dallas Cowboys,NYG,-4.0,...,0.0,24.0,17.0,1.0,2012.0,24.0,17.0,15.0,10.0,8.0
1,9/9/2012,2012,1.0,False,Arizona Cardinals,20.0,16.0,Seattle Seahawks,SEA,-3.0,...,0.0,16.0,20.0,1.0,2012.0,16.0,20.0,27.0,12.0,31.0
2,9/9/2012,2012,1.0,False,Chicago Bears,41.0,21.0,Indianapolis Colts,CHI,-10.0,...,0.0,21.0,41.0,1.0,2012.0,21.0,41.0,20.0,30.0,22.0
3,9/9/2012,2012,1.0,False,Cleveland Browns,16.0,17.0,Philadelphia Eagles,PHI,-9.5,...,0.0,17.0,16.0,1.0,2012.0,17.0,16.0,23.0,6.0,9.0
4,9/9/2012,2012,1.0,False,Denver Broncos,31.0,19.0,Pittsburgh Steelers,DEN,-2.0,...,0.0,19.0,31.0,1.0,2012.0,19.0,31.0,22.0,25.0,20.0


In [167]:
scores_and_rankings = pd.merge(scores_and_standings_all, preseason_rankings_df.add_suffix("_home"), 
                                how = "left", left_on = ['schedule_season', 'team_home'], 
                                right_on=['season_home', 'Team_home'])

scores_and_rankings_all = pd.merge(scores_and_rankings, preseason_rankings_df.add_suffix("_away"), 
                                how = "left", left_on = ['schedule_season', 'team_away'], 
                                right_on=['season_away', 'Team_away'])

scores_and_rankings_all.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,...,SOS_home,season_home,Team_away,Predictive_away,Home_away,Away_away,Last 5_away,In Div._away,SOS_away,season_away
0,9/5/2012,2012,1.0,False,New York Giants,17.0,24.0,Dallas Cowboys,NYG,-4.0,...,9,2012.0,Dallas Cowboys,20,20,20,19,19,20,2012.0
1,9/9/2012,2012,1.0,False,Arizona Cardinals,20.0,16.0,Seattle Seahawks,SEA,-3.0,...,18,2012.0,Seattle Seahawks,27,27,27,12,12,27,2012.0
2,9/9/2012,2012,1.0,False,Chicago Bears,41.0,21.0,Indianapolis Colts,CHI,-10.0,...,25,2012.0,Indianapolis Colts,21,21,21,13,13,21,2012.0
3,9/9/2012,2012,1.0,False,Cleveland Browns,16.0,17.0,Philadelphia Eagles,PHI,-9.5,...,22,2012.0,Philadelphia Eagles,4,4,4,8,8,4,2012.0
4,9/9/2012,2012,1.0,False,Denver Broncos,31.0,19.0,Pittsburgh Steelers,DEN,-2.0,...,7,2012.0,Pittsburgh Steelers,2,2,2,14,14,2,2012.0


In [170]:
# feature engineering

# Is the team bad? Preseason ranking used before week 8, otherwise, use the win percentage rank

scores_and_rankings_all['team_home_ranking'] = scores_and_rankings_all['Predictive_home'].where(
    scores_and_rankings_all['schedule_week'] <= 7, scores_and_rankings_all['rank_winpct_home']
)

scores_and_rankings_all['team_away_ranking'] = scores_and_rankings_all['Predictive_away'].where(
    scores_and_rankings_all['schedule_week'] <= 7, scores_and_rankings_all['rank_winpct_away']
)

scores_and_rankings_all['bad_team_involved'] = 

In [171]:
scores_and_rankings_all[['team_home_ranking', 'Predictive_home', 'schedule_week', 'rank_winpct_home']]

,team_home_ranking,Predictive_home,schedule_week,rank_winpct_home
0,9,9,1.0,27.0
1,18,18,1.0,16.0
2,25,25,1.0,11.0
3,22,22,1.0,21.0
4,7,7,1.0,5.0
...,...,...,...,...
2571,6.0,7,18.0,6.0
2572,14.0,21,18.0,14.0
2573,21.0,16,18.0,21.0
2574,30.0,25,18.0,30.0
